In [43]:
import os
import pickle
import json
import argparse

from tqdm import tqdm
import pandas as pd

In [44]:
def define_argparser():
    p = argparse.ArgumentParser()

    p.add_argument(
        '--load_path',
        default = './data/json',
        help="Path of file or directory to load original corpus."
    )
    p.add_argument(
        '--save_path',
        default = './data/raw',
        help="Path to save data."
    )
    p.add_argument(
        '--return_tsv',
        default = True,
        help="If not true, only pickle file will be saved."
    )

    config = p.parse_args(args=[])

    return config

In [45]:
config = define_argparser()

In [46]:
def json_to_tsv(file: str):
    cols = ['sentence_id', 'sentence', 'ne'] # 데이터 프레임 생성
    df = pd.DataFrame(columns=cols) # 데이터 프레임 생성
    id = 0

    with open(file) as f:
        DATA = json.loads(f.read()) # 파일을 열어서

    ne = []

    for i in range(0, len(DATA['annotations'])):
        if DATA['annotations'][i] == None:
            continue
        else:
            # print(DATA['annotations'][i][0])
            df.loc[id, 'sentence'] = DATA['annotations'][i][0]
            # print(DATA['annotations'][i][1]['entities'])
            # for entity in DATA['annotations'][i][1]['entities']:
            #     print(entity)

            # 엔티티 정보를 딕셔너리로 변환
            entity_dict = {}
            for index, (start, end, label) in enumerate(DATA['annotations'][i][1]['entities'], 1):
                # 문자열 슬라이싱을 사용하여 형태소 'form' 추출
                form = DATA['annotations'][i][0][start:end]
                entity_dict[index] = {
                    'form': form,
                    'label': label,
                    'begin': start,
                    'end': end
                }
            # print(entity_dict)
            ne.append(entity_dict)
            id += 1
    df['ne'] = ne

    return df

In [47]:
load_path = config.load_path # json 파일이 담긴 폴더 혹은 파일 그 자체
fn = os.path.split(config.load_path)[1] # config.load_path에서 파일명만 분리하여 fn 변수에 할당합니다. os.path.split() 함수는 경로를 디렉토리 부분과 파일명 부분으로 나눕니다. 이 중 파일명 부분만을 fn에 저장합니다.
if fn.rfind(".") > 0: # 파일명 fn에서 확장자를 제거합니다. rfind(".")는 파일명에서 마지막으로 나타나는 .의 위치를 찾습니다. 만약 .이 발견되면, 파일명에서 그 위치까지를 잘라서 확장자 없는 파일명을 fn으로 저장합니다.
    fn = fn[:fn.rfind(".")]
save_fn = os.path.join(config.save_path, fn) # 변환된 데이터의 결과물을 저장할 경로를 save_fn 변수에 할당합니다. config.save_path에 지정된 디렉토리 경로와 앞에서 만든 파일명 fn을 합쳐서 저장합니다.

print(f'load_path: {load_path}')
print(f'save_fn: {save_fn}')

load_path: ./data/json
save_fn: ./data/raw/json


In [48]:
if os.path.isdir(load_path): # load_path가 디렉토리인지 확인합니다. 만약 디렉토리라면 그 안에 있는 모든 파일을 반복적으로 처리합니다.
# os.listdir(load_path)는 load_path 디렉토리 안의 모든 파일 목록을 반환합니다.
    dfs = []
    for file in tqdm(os.listdir(load_path)): # tqdm을 사용해 파일 목록을 처리하면서 진행 상태를 시각적으로 표시합니다.
        df = json_to_tsv(os.path.join(load_path, file))
        dfs.append(df)
        # 각 파일을 json_to_tsv() 함수로 읽어들여, 그 결과를 데이터프레임 df로 변환한 뒤 dfs 리스트에 추가합니다.
    data = pd.concat(dfs) # 모든 파일의 데이터프레임을 pd.concat(dfs)로 하나의 데이터프레임 data로 결합합니다.
else:
    data = json_to_tsv(load_path) # 만약 load_path가 파일이라면, 단일 파일을 json_to_tsv() 함수로 처리해 data에 할당합니다.

100%|██████████| 10/10 [00:00<00:00, 207.65it/s]


In [49]:
data

,sentence_id,sentence,ne
0,NaN,근거가 없을 뿐만 아니라 이러한 연차휴가근로수당이라는 것은 근로기준법에서 특별히 정...,{}
1,NaN,"""한편 """"IVY""""는 """"담쟁이 덩굴""""을 뜻하는 것으로서 """"HOUSE""""를 단...",{}
2,NaN,"""한편 """"IVY""""는 """"담쟁이 덩굴""""을 뜻하는 것으로서 위와 같이 포괄적,일반...",{}
3,NaN,피고법인이 설치하여 운영하는 목원대학의 법학과에 입학하여 1987학년도 1학기(4학...,"{1: {'form': '피고법인', 'label': 'PS_ASSAILANT', ..."
4,NaN,1이 담당교수과목인 민사연습과 법학특강(각 3학점)을 비롯한 4과목(합계 12학점)...,"{1: {'form': '3학점', 'label': 'QT_SCORE', 'begi..."
...,...,...,...
13,NaN,제관원으로 각 입사하여 피고 양회 동해공장 기계과 소속으로 시멘트 제조기계의 보수 ...,"{1: {'form': '제관원', 'label': 'JB', 'begin': 0,..."
14,NaN,내게 되어 그 충격으로 협착된 경추내 신경에 손상이 감으로써 팔다리가 마비되는 부상...,"{1: {'form': '협착된 경추내 신경에 손상이 감', 'label': 'PC..."
15,NaN,건물의 콘크리트 벽을 들이 받는 사고를 일으켜 그 충격으로 팔다리가 마비되는 부상(...,"{1: {'form': '건물의 콘크리트 벽', 'label': 'LC_OTHER_..."
16,NaN,"길이 3센티미터, 왼쪽 가슴 부분에 길이 6센티미터의 상처가 나도록 찔렀고 그 가슴...","{1: {'form': '왼쪽 가슴 부분', 'label': 'BP', 'begin..."


In [50]:
with open(save_fn+'.pickle', "wb") as f: # 변환된 데이터를 피클(pickle) 파일로 저장합니다. 파일명은 save_fn에 .pickle 확장자를 붙인 것입니다. 
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
    # pickle.dump() 함수는 데이터를 피클 형식으로 직렬화하고, protocol=pickle.HIGHEST_PROTOCOL을 사용해 가능한 최고의 프로토콜을 사용하여 저장합니다.

if config.return_tsv: # config.return_tsv가 True인 경우, 데이터프레임 data를 TSV 파일로 저장합니다.
    data.to_csv(save_fn+'.tsv', sep='\t', index=False)
    # 파일명은 save_fn에 .tsv 확장자를 붙인 것이며, 탭(\t)으로 구분된 형식으로 저장됩니다.
    # index=False로 설정하여, 데이터프레임의 인덱스는 파일에 포함되지 않도록 합니다.